# Transfer learning notebook

Dans ce notebook, je vais prendre un réseau de neurones pré-entraînés, freeze des layers et entraîner la dernière couche pour prédire si un mot est un nom de personne ou non.

Il se décompose en 4 étapes:
- *Data:* Load data "MultiNERD" (données wikipedia labelisées)
- *Feature:* Créer un jeu de données X, y pour dire quels mots sont des noms de personne, formatté pour HuggingFace.<br/>
Je créé un jeu "train" (pour apprendre), "dev" (pour évaluer le modèle pendant l'apprentissage) et "test" pour mesurer l'accuracy (ou autre mesure) une fois toutes mes optimisations faites.
- *Model:* Prendre un réseau de neurones, freeze les layers
- *Train:* L'entraîner

In [114]:
import csv
import numpy as np
import torch
import transformers

In [115]:
# model_name = "distilbert/distilbert-base-cased"
model_name = "almanach/camembert-base"

## Data
### MultiNERD data

Ce dataset est un text avec des catégories assez fines (dont nom de personne).<br>
Il est disponible [sur ce lien](https://github.com/Babelscape/multinerd)

In [116]:
with open("../data/raw/train_multinerd_fr.tsv") as f:
    rows = list(line.strip().split("\t") for line in f)

rows[:10]

[['0', 'Il', 'O'],
 ['1', 'est', 'O'],
 ['2', 'incarné', 'O'],
 ['3', 'par', 'O'],
 ['4',
  'Austin',
  'B-PER',
  'bn:02525192n',
  'Q4204710',
  '7345300',
  'Austin_Stowell',
  'Austin Stowell est un acteur américain né le 24 décembre 1984 à Kensington dans le Connecticut.',
  'https://upload.wikimedia.org/wikipedia/commons/9/95/Austin_Stowell-DolphinTale.jpg'],
 ['5', 'Stowell', 'I-PER'],
 ['6', '.', 'O'],
 [''],
 ['0', 'c’', 'O'],
 ['1', 'est', 'O']]

## Feature

### Créer le jeu X (mot), y (est-ce un nom de personne)

In [117]:
def make_labelled_sentences(tagged_words):
    # Joining words until we meet a dot
    # Word's label is 1 if 'PER' is in its tag
    X = []
    y = []

    this_word = []
    this_labels = []
    for tagged_word in tagged_words:
        if len(tagged_word) < 3:
            # not a tagged word
            continue
        word = tagged_word[1]
        tag = tagged_word[2]

        if word == '.':
            X.append(this_word)
            y.append(this_labels)

            this_word = []
            this_labels = []
        else:
            this_word.append(word)
            this_labels.append(1 * tag.endswith("PER"))

    return X, y

In [118]:
# Quand on essaie un modèle / framework
# PAS la peine de faire tourner sur tout le dataset
# On peut prendre un sous-ensemble et vérifier que le code tourne
sentences, labels = make_labelled_sentences(rows[:100_000])

In [119]:
sentences[2], labels[2]

(['À', 'Madras', ',', 'le'], [0, 0, 0, 0])

In [120]:
from sklearn.model_selection import train_test_split

In [121]:
sentences_training, sentences_test, labels_training, labels_test = train_test_split(
    sentences,
    labels,
    test_size=0.2,
    random_state=42,
)

In [122]:
sentences_train, sentences_dev, labels_train, labels_dev = train_test_split(
    sentences_training,
    labels_training,
    test_size=0.2,
    random_state=42,
)

### Transformer ce jeu X, y en données pour un modèle HuggingFace

J'utilise le tokenizer pour transformer les mots en tokens.
Et j'applique les labels:
- 1 si le mot est un nom de personne ET ce token est le 1er token du mot.
- 0 si le mot n'est pas un nom de personne ET ce token est le 1er token du mot.
- -100 sinon (token pas à prédire, convention HuggingFace)

In [123]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

In [124]:
def tokenize_and_align_labels(sentences, ner_tags):
    tokenized_inputs = tokenizer(
        sentences,
        truncation=True,
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(ner_tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

In [125]:
tokenized_train = tokenize_and_align_labels(sentences_train, labels_train)

In [126]:
tokenized_test = tokenize_and_align_labels(sentences_test, labels_test)

In [127]:
from datasets import Dataset

dataset_train = Dataset.from_dict(tokenized_train)
dataset_test = Dataset.from_dict(tokenized_test)

In [128]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Model
## V1: learning only last layer

In [129]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=2
)
# model = model.to("cuda")

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at almanach/camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [130]:
for name, _ in model.base_model.named_parameters():
  print(name)

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

In [131]:
for name, param in model.base_model.named_parameters():
  param.requires_grad = False

for name, param in model.base_model.named_parameters():
    if (
        any(layer_name in name for layer_name in ["layer.11"])
        and any(layer_type in name for layer_type in ["weight", "bias"])
        and "ffn.lin" in name
    ):
        param.requires_grad = True

# Train 

## Méthode d'évaluation du modèle

Calculant accuracy, log loss, etc

In [132]:
import numpy as np
import evaluate

seqeval = evaluate.load("seqeval")

labels = [0, 1]
label_list = ["0", "1"]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Train du modèle

In [137]:
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipykernel_1218412/1595583786.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


AcceleratorError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# A vous de jouer

Créer la fonction prenant les prédictions niveau token, et me donnant prédictions niveau mot.
*Note:* Dans ce genre de cas, prenez des mots bizarres, qui font plusieurs tokens. <br/>
De cette façon, vous évitez des erreurs. <br/>
Si vous ne prenez que des mots simples ["bonjour", "monsieur", "jean"] (qui ne font qu'un token), vous pouvez croire que votre code marche, mais qu'il casse dès qu'un mot fait plusieurs tokens

In [87]:
words = ["Hellotapasci", "mister", "Bondaboliot", "goodbye"]
wanted_labels = [0, 0, 1, 0]
def predict_at_word_level(words, model, tokenizer):
    # Tokeniser les mots
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to("cpu")
    tokenized_inputs = tokenizer(
        words,
        truncation=True,
        is_split_into_words=True,
        return_tensors="pt"
    )
    
    # Faire une prédiction avec le modèle
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
        print(outputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    
    # Mapper les prédictions des tokens aux mots
    word_ids = tokenized_inputs.word_ids()
    word_predictions = []
    previous_word_idx = None
    
    for i, word_idx in enumerate(word_ids):
        if word_idx is not None and word_idx != previous_word_idx:
            # Prendre la prédiction du premier token de chaque mot
            word_predictions.append(int(predictions[0][i]))
        previous_word_idx = word_idx
    
    return word_predictions
    

In [ ]:
predict_at_word_level(words, model, tokenizer)

In [89]:
# Prédire sur le dataset France Inter
import pandas as pd
import ast
from sklearn.metrics import accuracy_score, classification_report

# Charger le dataset France Inter
df = pd.read_csv('../data/raw/train_v3.csv')
print(f"Dataset chargé avec {len(df)} exemples")
print(df.head())


Dataset chargé avec 999 exemples
                                          video_name  \
0  Le Barbecue Disney - La chanson de Frédéric Fr...   
1  Le Roi et l'Oiseau - La Chronique de Christine...   
2  L'amour du lac - La chronique d'Hippolyte Gira...   
3  La fille de la piscine de Léa Tourret - La chr...   
4  "Le soleil va moins faire son malin quand Jean...   

                                             is_name  \
0                        [0, 0, 0, 0, 0, 0, 0, 1, 1]   
1                     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]   
2                     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]   
3         [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1]   
4  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...   

                                              tokens  
0  ["Le", "Barbecue", "Disney", "-", "La", "chans...  
1  ["Le", "Roi", "et", "l'Oiseau", "-", "La", "Ch...  
2  ["L'", "amour", "du", "lac", "-", "La", "chron...  
3  ["La", "fille", "de", "la", "piscine", "de", "...  
4  ["\"Le", "solei

In [90]:
# Traitement des données et prédictions
y_true_all = []
y_pred_all = []

# Prendre un échantillon pour tester (les 100 premières lignes)
sample_df = df.head(100)

for idx, row in sample_df.iterrows():
    try:
        # Parser les labels et tokens
        labels = ast.literal_eval(row['is_name'])
        tokens = ast.literal_eval(row['tokens'])
        
        # Vérifier que les longueurs correspondent
        if len(labels) != len(tokens):
            print(f"Ligne {idx}: longueurs différentes - labels: {len(labels)}, tokens: {len(tokens)}")
            continue
            
        # Faire la prédiction
        predictions = predict_at_word_level(tokens, model, tokenizer)
        
        # Vérifier que les prédictions ont la bonne longueur
        if len(predictions) != len(labels):
            print(f"Ligne {idx}: prédictions de longueur différente - prédictions: {len(predictions)}, labels: {len(labels)}")
            continue
            
        y_true_all.extend(labels)
        y_pred_all.extend(predictions)
        
    except Exception as e:
        print(f"Erreur ligne {idx}: {e}")
        continue

print(f"\nNombre total d'exemples traités: {len(y_true_all)}")
print(f"Accuracy: {accuracy_score(y_true_all, y_pred_all):.4f}")
print("\nRapport de classification:")
print(classification_report(y_true_all, y_pred_all, target_names=['Pas un nom', 'Nom de personne']))



Nombre total d'exemples traités: 1171
Accuracy: 0.9718

Rapport de classification:
                 precision    recall  f1-score   support

     Pas un nom       0.97      1.00      0.98       979
Nom de personne       1.00      0.83      0.91       192

       accuracy                           0.97      1171
      macro avg       0.98      0.91      0.94      1171
   weighted avg       0.97      0.97      0.97      1171



In [91]:
# Afficher quelques exemples de prédictions
print("Exemples de prédictions:\n")
for idx in range(min(5, len(sample_df))):
    row = sample_df.iloc[idx]
    try:
        labels = ast.literal_eval(row['is_name'])
        tokens = ast.literal_eval(row['tokens'])
        predictions = predict_at_word_level(tokens, model, tokenizer)
        
        if len(predictions) == len(labels):
            print(f"Exemple {idx + 1}: {row['video_name']}")
            print("Mots        :", tokens)
            print("Vrai labels :", labels)
            print("Prédictions :", predictions)
            print("Correct     :", [1 if p == l else 0 for p, l in zip(predictions, labels)])
            print()
    except:
        continue


Exemples de prédictions:

Exemple 1: Le Barbecue Disney - La chanson de Frédéric Fromet
Mots        : ['Le', 'Barbecue', 'Disney', '-', 'La', 'chanson', 'de', 'Frédéric', 'Fromet']
Vrai labels : [0, 0, 0, 0, 0, 0, 0, 1, 1]
Prédictions : [0, 0, 0, 0, 0, 0, 0, 1, 1]
Correct     : [1, 1, 1, 1, 1, 1, 1, 1, 1]

Exemple 2: Le Roi et l'Oiseau - La Chronique de Christine Gonzalez
Mots        : ['Le', 'Roi', 'et', "l'Oiseau", '-', 'La', 'Chronique', 'de', 'Christine', 'Gonzalez']
Vrai labels : [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
Prédictions : [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
Correct     : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Exemple 3: L'amour du lac - La chronique d'Hippolyte Girardot
Mots        : ["L'", 'amour', 'du', 'lac', '-', 'La', 'chronique', "d'", 'Hippolyte', 'Girardot']
Vrai labels : [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
Prédictions : [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Correct     : [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]

Exemple 4: La fille de la piscine de Léa Tourret - La chronique de Juliette Arnaud
Mots     